# Intro/Imports
This is an adaptation of nnExpPreds that will allow for adaptive changing of starting conditions, generating a set of model predictions to match.

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import pathlib as path
import copy
import random

# Model Import

In [12]:
model = torch.load('models/model.pt')
model.eval()
#loss/optimizer
loss_fn = nn.MSELoss() #Mean Squared Error
optimizer = optim.Adam(model.parameters(), lr=0.0005)

#scalers
mmscalerX = joblib.load('models/mmscalerX.pkl')
mmscalerY = joblib.load('models/mmscalerY.pkl')


# Initial Condition Prep
The prediction loop works by iterating thru a dataframe; feeding the current row through the model, and replacing the values of the next row with the results from the model. 
- X is a dataframe with rows: C_X, C_N, C_L, F_in, C_N_in, I0, and Time
- Y is a dataframe with rows C_X, C_N, C_L

In [13]:
XTimes = np.linspace(0, 150, 200)
XDF = pd.DataFrame(XTimes, columns=['Time'])
#generate empty columns
XDF['C_X'] = np.zeros(200)
XDF['C_N'] = np.zeros(200)
XDF['C_L'] = np.zeros(200)
XDF['F_in'] = np.zeros(200)
XDF['C_N_in'] = np.zeros(200)
XDF['I0'] = np.zeros(200)

#set initial conditions by changing these vals
C_X_init = 1.5
C_N_init = 0.7
C_L_init = 0.0
F_in_init = 0.009
C_N_in_init = 11
I0_init = 136
#write init conditions to df
XDF['C_X'][0] = C_X_init
XDF['C_N'][0] = C_N_init
XDF['C_L'][0] = C_L_init
XDF['F_in'][0] = F_in_init
XDF['C_N_in'][0] = C_N_in_init
XDF['I0'][0] = I0_init
